# Notebook for testing and exploring objectives in tesser

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
from scipy import optimize
from tesser import network
from tesser import util
from tesser import sr
from tesser import fit

In [2]:
data_dir = "/home/rodrigo/Dropbox/tesser_successor/Data/"

In [3]:
x = util.load_struct_subject(data_dir, 101)

In [4]:
%timeit sr.clearn_sr(x, 0.5, 0.5,21)

131 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
%load_ext Cython

convert function to cython

In [47]:
induct_all = util.load_induct(data_dir)
struct_all = util.load_struct(data_dir)

subj_filter = f'SubjNum == {101}'
induct_df = induct_all.query(subj_filter)
struct_df = struct_all.query(subj_filter)

n_states = len(np.unique(struct_df.objnum))

# get community matrix
net = network.temp_node_info()
comm = 1 - distance.squareform(distance.pdist(net['comm'][:, None], 'hamming'))

now to attempt fixing fitting times

In [53]:
fixed = {'w': 1.0}
var_names = ['gamma', 'alpha', 'tau']
var_bounds = {'alpha': [0, 1], 'gamma': [0, 1], 'tau': [0, 10]}
fit_induct(struct_all, induct_all, fixed, var_names, var_bounds, 
           n_states=n_states, verbose=False, comm=comm)

({'w': 1.0,
  'gamma': 0.9915182972903186,
  'alpha': 0.04573502900941932,
  'tau': 1.0664064743190262},
 -1053.598065597301)

In [54]:
print(fit.cmain(struct_df, induct_df, 0.99, 0.02, 1.0, 1.0,n_states, True, comm=comm))

(-28.89193189195182, <MemoryView of 'ndarray' at 0x7f1cdad67890>)


In [55]:
import time
start_time =time.time()
fit.cmain(struct_df, induct_df, 0.5, 0.5, 0.5, 1.0, n_states, True, comm=comm)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0020296573638916016 seconds ---


In [56]:
import time
start_time =time.time()
fit.get_induction_log_likelihood_hybrid(struct_df, induct_df, 0.5, 0.5, 0.5, 1.0)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04805946350097656 seconds ---


In [36]:
%timeit fit.cmain(struct_df, induct_df, 0.5, 0.5, 0.5, 1.0, n_states, True, comm=comm)

1.39 ms ± 55.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [57]:
%timeit fit_induct(struct_all, induct_all, fixed, var_names, var_bounds,n_states=n_states, verbose=False, comm=comm)

1min 2s ± 8.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
